<a href="https://colab.research.google.com/github/mona-jha/python/blob/main/gis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install required tools
!apt install -y gdal-bin
!pip install rasterio
!pip install GDAL==$(gdal-config --version) numpy matplotlib


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gdal-bin is already the newest version (3.6.4+dfsg-1~jammy0).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
!pip install -U gdown
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import gdown

folder_id = "14Mdf3BP0nI49hsJLtXFP4w-ZCYsMPSE-"
gdown.download_folder(f"https://drive.google.com/drive/folders/{folder_id}", quiet=False, use_cookies=False)


In [ ]:
import os
from glob import glob
import rasterio
import numpy as np
import subprocess

# ✅ Update this to your actual folder with DEM files
dem_folder = "/content/final"
dem_files = glob(f"{dem_folder}/*.tif")

print(f" Found {len(dem_files)} DEM files:")
for f in dem_files:
    print("-", os.path.basename(f))


In [ ]:
normalized_files = []

for fp in dem_files:
    base = os.path.basename(fp).replace(".tif", "")

    # Step 1: Reproject to EPSG:4326 (if needed)
    with rasterio.open(fp) as src:
        if src.crs is None or src.crs.to_epsg() != 4326:
            reproj_fp = f"/content/{base}_reproj.tif"
            subprocess.run([
                "gdalwarp", "-t_srs", "EPSG:4326", "-r", "bilinear", "-overwrite", fp, reproj_fp
            ])
        else:
            reproj_fp = fp

    # Step 2: Normalize elevation values
    with rasterio.open(reproj_fp) as src:
        data = src.read(1)
        profile = src.profile
        nodata = src.nodata or -9999

        mask = (data != nodata)
        valid = data[mask]

        data_norm = np.full_like(data, nodata, dtype=np.float32)
        if valid.size > 0 and valid.max() != valid.min():
            data_norm[mask] = (valid - valid.min()) / (valid.max() - valid.min())

        # Save normalized raster
        norm_fp = f"/content/{base}_norm.tif"
        profile.update(dtype='float32', nodata=nodata)

        with rasterio.open(norm_fp, "w", **profile) as dst:
            dst.write(data_norm, 1)

        normalized_files.append(norm_fp)

print(f" Reprojected and normalized: {len(normalized_files)} files")


In [ ]:
output_path = "/content/merged_dem_normalized.tif"

merge_cmd = [
    "gdal_merge.py",
    "-o", output_path,
    "-of", "GTiff",
    "-a_nodata", "-9999"
] + normalized_files

subprocess.run(merge_cmd)

print(f" Merged DEM saved to: {output_path}")


# Save DEM File

In [ ]:
import shutil
import os

# Create a folder in your Google Drive (if needed)
target_folder = "/content/drive/My Drive/dem_outputs"
os.makedirs(target_folder, exist_ok=True)

# Set target path and copy the file
source_file = "/content/merged_dem_normalized.tif"
target_file = os.path.join(target_folder, "merged_dem_normalized.tif")

shutil.copy(source_file, target_file)

print(f" File saved to: {target_file}")


#analysis

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

# Replace this path with your actual DEM path
dem_path = "/content/merged_dem_normalized.tif"

with rasterio.open(dem_path) as src:
    dem = src.read(1)
    transform = src.transform
    crs = src.crs
    nodata = src.nodata

print(f"DEM shape: {dem.shape}, CRS: {crs}")


# Elevation Profile Along a Line

In [ ]:
from osgeo import gdal, ogr, osr
import numpy as np
import matplotlib.pyplot as plt

# Paths
dem_path = "/content/merged_dem_normalized.tif"
line_shp = "/content/profile_line.shp"  # This is a polyline shapefile


In [ ]:
from osgeo import gdal, ogr, osr
import numpy as np
import matplotlib.pyplot as plt

# Paths
dem_path = "/content/merged_dem_normalized.tif"
line_shp = "/content/profile_line.shp"  # This is a polyline shapefile


In [ ]:
# Open DEM
dem_ds = gdal.Open(dem_path)
band = dem_ds.GetRasterBand(1)
gt = dem_ds.GetGeoTransform()
nodata = band.GetNoDataValue()

# Get projection of DEM
dem_srs = osr.SpatialReference()
dem_srs.ImportFromWkt(dem_ds.GetProjection())

# Open Line shapefile
shp_ds = ogr.Open(line_shp)
layer = shp_ds.GetLayer()
line = layer.GetNextFeature().GetGeometryRef()

# Reproject line if needed
line_srs = layer.GetSpatialRef()
if not line_srs.IsSame(dem_srs):
    transform = osr.CoordinateTransformation(line_srs, dem_srs)
    line.Transform(transform)

# Sample N points along the line
N = 200
dist = line.Length() / (N - 1)
points = [line.Value(i * dist) for i in range(N)]

# Sample elevation from DEM
elevations = []
for point in points:
    x, y, _ = point.GetPoint()
    col = int((x - gt[0]) / gt[1])
    row = int((y - gt[3]) / gt[5])
    value = band.ReadAsArray(col, row, 1, 1)[0][0]
    elevations.append(np.nan if value == nodata else value)


In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(elevations, color='green')
plt.title("Elevation Profile Along the Line")
plt.xlabel("Distance along line (arbitrary units)")
plt.ylabel("Elevation")
plt.grid(True)
plt.show()


In [ ]:
from osgeo import ogr

line_path = "/content/profile_line.shp"

driver = ogr.GetDriverByName("ESRI Shapefile")
ds = driver.CreateDataSource(line_path)
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)  # Or same EPSG as your DEM
layer = ds.CreateLayer("line", srs, ogr.wkbLineString)

line = ogr.Geometry(ogr.wkbLineString)
line.AddPoint(77.5, 12.9)
line.AddPoint(77.7, 13.0)

feature_defn = layer.GetLayerDefn()
feature = ogr.Feature(feature_defn)
feature.SetGeometry(line)
layer.CreateFeature(feature)

ds = None  # Close
print("✅ Line shapefile created:", line_path)


#Contour Analysis

In [ ]:
# Path to your DEM file
dem_path = "/content/merged_dem_normalized.tif"
dem_ds = gdal.Open(dem_path)
band = dem_ds.GetRasterBand(1)
nodata = band.GetNoDataValue() or -9999

In [ ]:
from osgeo import ogr

# Output shapefile path
contour_shp = "/content/contours.shp"

# Create output shapefile
driver = ogr.GetDriverByName("ESRI Shapefile")
if driver is None:
    raise RuntimeError("Shapefile driver not available")

# Delete existing if present
driver.DeleteDataSource(contour_shp)
ds = driver.CreateDataSource(contour_shp)
layer = ds.CreateLayer("contours", srs=dem_ds.GetProjectionRef(), geom_type=ogr.wkbLineString)

# Add elevation attribute field
field_defn = ogr.FieldDefn("ELEV", ogr.OFTReal)
layer.CreateField(field_defn)

# Generate contours every 10 units (you can change)
gdal.ContourGenerate(
    band,              # DEM band
    10,                # contour interval (e.g., 10m)
    0,                 # base contour value
    [],                # fixed levels (empty = use interval)
    0,                 # use no-data (0 = false)
    nodata,            # no-data value
    layer,             # output layer
    0,                 # ID field index (not used)
    0                  # elevation field index (0 = ELEV)
)

ds.Destroy()
print("✅ Contour shapefile created:", contour_shp)


Save

In [ ]:
from google.colab import drive
import shutil

drive.mount('/content/drive')

# Save contour shapefile to Drive
shutil.copy("/content/contours.shp", "/content/drive/My Drive/contours.shp")
shutil.copy("/content/contours.dbf", "/content/drive/My Drive/contours.dbf")
shutil.copy("/content/contours.shx", "/content/drive/My Drive/contours.shx")
shutil.copy("/content/contours.prj", "/content/drive/My Drive/contours.prj")

print("✅ Contours saved to Google Drive.")


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show

# Load contour lines
contours = gpd.read_file(contour_shp)

# Plot DEM and overlay contours
with rasterio.open(dem_path) as src:
    fig, ax = plt.subplots(figsize=(10, 8))
    show(src, ax=ax, cmap='terrain', title="Contours over DEM")
    contours.plot(ax=ax, linewidth=0.5, color='black')
    plt.show()


# Slope Analysis  

In [ ]:
from osgeo import gdal

# Input DEM
dem_path = "/content/merged_dem_normalized.tif"

# Output slope raster
slope_path = "/content/slope_degrees.tif"


In [ ]:
# Compute slope in degrees
gdal.DEMProcessing(
    slope_path,        # Output path
    dem_path,          # Input DEM
    "slope",           # Operation
    options=gdal.DEMProcessingOptions(slopeFormat="degree")
)

print(f"✅ Slope (degrees) saved to: {slope_path}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

ds = gdal.Open(slope_path)
slope_array = ds.ReadAsArray()

plt.figure(figsize=(10, 6))
plt.imshow(slope_array, cmap='terrain')
plt.title("Slope Map (degrees)")
plt.colorbar(label="Degrees")
plt.axis('off')
plt.show()


In [ ]:
from google.colab import drive
import shutil

drive.mount('/content/drive')

shutil.copy(slope_path, "/content/drive/My Drive/slope_degrees.tif")
print(" Slope saved to Google Drive")


# Aspect Analysis

In [ ]:
# Output aspect map
aspect_path = "/content/aspect.tif"

# Compute aspect using GDAL
subprocess.run([
    "gdaldem", "aspect", dem_path, aspect_path,
    "-of", "GTiff"
])

# Visualize aspect
with rasterio.open(aspect_path) as src:
    aspect = src.read(1)

plt.figure(figsize=(8, 6))
plt.imshow(aspect, cmap='hsv')
plt.title("Aspect Map (0° = North)")
plt.colorbar(label="Azimuth (°)")
plt.axis("off")
plt.show()


# Hydrological Analysis

In [ ]:
dem_path = "/content/merged_dem_normalized.tif"
filled_dem = "/content/filled_dem.tif"

!gdal_fillnodata.py -of GTiff -co COMPRESS=DEFLATE -co TILED=YES \
  {dem_path} {filled_dem}


In [ ]:
import richdem as rd

# Load filled DEM
rdem = rd.LoadGDAL(filled_dem)

# Compute flow direction using D8 algorithm
flow_dir = rd.FlowDirD8(rdem)
flow_dir_path = "/content/flow_direction.tif"
rd.SaveGDAL(flow_dir_path, flow_dir)

print(" Flow direction saved:", flow_dir_path)


In [ ]:
# Compute flow accumulation
flow_acc = rd.FlowAccumD8(rdem, in_place=False)
flow_acc_path = "/content/flow_accumulation.tif"
rd.SaveGDAL(flow_acc_path, flow_acc)

print(" Flow accumulation saved:", flow_acc_path)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Convert to log scale for visualization
acc_data = np.array(flow_acc, dtype=np.float32)
acc_data[acc_data <= 0] = np.nan

plt.figure(figsize=(10, 6))
plt.imshow(np.log1p(acc_data), cmap='Blues')
plt.title("Flow Accumulation (log scale)")
plt.axis("off")
plt.colorbar(label="Log(Accumulation)")
plt.show()


In [ ]:
# Thresholding high accumulation zones (tune value as needed)
flood_mask = acc_data > 500  # Try 1000 or 2000 for smaller rivers

plt.figure(figsize=(10, 6))
plt.imshow(flood_mask, cmap='Reds')
plt.title("Potential Flooding Zones / River Crossings")
plt.axis("off")
plt.show()


In [ ]:
# Create output folder in Drive
output_folder = "/content/drive/My Drive/hydrology_results"
os.makedirs(output_folder, exist_ok=True)

# Save files
shutil.copy(flow_dir_path, os.path.join(output_folder, "flow_direction.tif"))
shutil.copy(flow_acc_path, os.path.join(output_folder, "flow_accumulation.tif"))
shutil.copy(filled_dem, os.path.join(output_folder, "filled_dem.tif"))

print(f"All outputs saved to Google Drive: {output_folder}")

# Surface Analysis

In [ ]:
from osgeo import gdal
import numpy as np

# Open DEM
dem_path = "/content/merged_dem_normalized.tif"
dem_ds = gdal.Open(dem_path)
dem_array = dem_ds.ReadAsArray().astype(np.float32)
geotransform = dem_ds.GetGeoTransform()
projection = dem_ds.GetProjection()
nodata = dem_ds.GetRasterBand(1).GetNoDataValue()

# Mask NoData
dem_array[dem_array == nodata] = np.nan
print(" DEM loaded:", dem_array.shape)


Ruggedness Index (TRI)

In [ ]:
from osgeo import gdal_array

def compute_tri_gdal(dem_array):
    padded = np.pad(dem_array, 1, mode='edge')
    tri_array = np.empty_like(dem_array, dtype=np.float32)

    rows, cols = dem_array.shape
    for i in range(rows):
        for j in range(cols):
            center = padded[i+1, j+1]
            window = padded[i:i+3, j:j+3]
            diffs = np.abs(window - center)
            tri_array[i, j] = np.nanmean(diffs)

    return tri_array

tri = compute_tri_gdal(dem_array)


In [ ]:
#Flat and Rough Terrain
# Define thresholds
rough_threshold = 0.05
flat_threshold = 0.01

rough_mask = (tri > rough_threshold).astype(np.uint8)
flat_mask = (tri < flat_threshold).astype(np.uint8)


In [ ]:
def save_array_as_tiff(array, out_path, reference_ds, dtype=gdal.GDT_Float32):
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(out_path, reference_ds.RasterXSize, reference_ds.RasterYSize, 1, dtype)
    out_ds.SetGeoTransform(reference_ds.GetGeoTransform())
    out_ds.SetProjection(reference_ds.GetProjection())
    out_ds.GetRasterBand(1).WriteArray(array)
    out_ds.GetRasterBand(1).SetNoDataValue(np.nan)
    out_ds.FlushCache()
    out_ds = None

save_array_as_tiff(tri, "/content/terrain_ruggedness.tif", dem_ds)
save_array_as_tiff(flat_mask, "/content/flat_zones.tif", dem_ds, dtype=gdal.GDT_Byte)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.imshow(tri, cmap='inferno')
plt.title("Terrain Ruggedness Index (TRI)")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(flat_mask, cmap='Greens')
plt.title("Flat Terrain Zones")
plt.axis('off')

plt.tight_layout()
plt.show()
